# Trim Vectors

In [3]:
# load models

from poems_model import PoemsModel

poems_data_file = './data/poems_model.pickle'
w2v_file = 'c:/data/ruscorpora_1_300_10.bin.gz'

pm = PoemsModel(poems_data_file, w2v_file)

2017-05-27 22:35:45,321 : INFO : loading projection weights from c:/data/ruscorpora_1_300_10.bin.gz


loading w2v_model...


2017-05-27 22:35:52,718 : INFO : loaded (161301, 300) matrix from c:/data/ruscorpora_1_300_10.bin.gz


word2vec model 'c:/data/ruscorpora_1_300_10.bin.gz' loaded
reading pickle poems model...
vectorizing model...
model is loaded


In [95]:
# reload PoemsModel class
import sys
del sys.modules['poems_model']
from poems_model import PoemsModel

2017-05-27 22:34:15,987 : INFO : Loading dictionaries from C:\Anaconda3\lib\site-packages\pymorphy2_dicts\data
2017-05-27 22:34:16,056 : INFO : format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168


In [ ]:
import numpy as np
from numpy import linalg as la

%timeit(np.sqrt((pm.w2v.syn0[0] ** 2).sum()))
%timeit(la.norm(pm.w2v.syn0[0]))

%timeit(la.norm(pm.w2v.syn0, axis=1))


In [24]:
from gensim.models.keyedvectors import KeyedVectors
import numpy as np
from numpy import linalg as la 

def trim_by_suffix(w2v: KeyedVectors, suffix='', factor=1.0):
    if suffix:
        for word in w2v.vocab:
            if suffix in word:
                v_index = w2v.vocab[word].index
                w2v.syn0[v_index] *= factor

                
def trim_vectors_by_vocab(pm: PoemsModel, factor=1.0):
    for word in pm.w2v.vocab:
        if not word in pm.vocab:
            v_index = pm.w2v.vocab[word].index
            pm.w2v.syn0[v_index] *= factor

def trim_vectors_not_in_list(pm: PoemsModel, focus_words: list, factor=1.0):
    for word in pm.w2v.vocab:
        if not word in focus_words:
            v_index = pm.w2v.vocab[word].index
            pm.w2v.syn0[v_index] *= factor
    

def normalize_vectors(w2v: KeyedVectors):
    for i in range(w2v.syn0.shape[0]):
        w2v.syn0[i] /= np.sqrt((w2v.syn0[i] ** 2).sum())


In [34]:
trim_by_suffix(pm.w2v, suffix='_VERB', factor=0.7)
trim_by_suffix(pm.w2v, suffix='_ADJ', factor=0.5)
la.norm(pm.w2v.syn0, axis=1)

array([ 1.        ,  1.00000012,  0.70000005, ...,  1.        ,
        1.        ,  0.69999999], dtype=float32)

In [13]:
trim_vectors_by_vocab(pm, factor=0.7)
la.norm(pm.w2v.syn0, axis=1)    

array([ 1.        ,  0.99999988,  0.25      , ...,  1.        ,
        0.99999994,  0.25      ], dtype=float32)

In [43]:
query = 'иллюзия'

words = [w[0] for w in pm.most_similar(query, topn=10000)]

trim_vectors_not_in_list(pm, words, factor=0.7)
la.norm(pm.w2v.syn0, axis=1)    

array([ 0.69999999,  0.70000005,  0.70000005, ...,  0.69999999,
        0.69999999,  1.        ], dtype=float32)

In [45]:
normalize_vectors(pm.w2v)
la.norm(pm.w2v.syn0, axis=1)

array([ 1.        ,  0.99999994,  0.99999994, ...,  1.        ,
        1.        ,  1.        ], dtype=float32)

In [ ]:
pm.most_similar(query, topn=50)

In [46]:
for poem in pm.similar_poems(query, topn=10): 
    print(f'{poem[0]}\t{poem[1]}')

проста как боль легка как правда
неосязаема как страх
скучна как мир пуста как память
твоя несбывшаяся жизнь
	0.2587770539915357
нам код коррекции ошибок
записан в глубину души
чтоб мы могли картину мира
из ложных данных извлекать
	0.2463622117742765
никто давно не верит в сказки
про бога и грядущий рай
вот апокалипсис немного
ещё способен нас увлечь
	0.24087915624377793
давайте верить в справедливость
как верят в бога в чудеса
или в какую то другую
несуществующую хрень
	0.23906972380797495
в альтернативном мире люди
не умирают никогда
и экзистенциально верят
в отсутствие альтернатив
	0.237362495795455
есть сверхспособности и в боли
способность есть способность спать
чудовищная ясность мысли
невосприимчивость к дождю
	0.23231856011251084
я идиот а это значит
весь мир красив и разноцветн
все люди мудры и прекрасны
а все кто умерли в раю
	0.23190658801215316
куда нас сон всегда приводит
в унылый серый плоский мир
где нет возможности падений
и творческих альтернатив
	0.23051201112138486
в